# Construção de vocabulário

* Tokenizando textos em palavras e n-gramas (tokens)

* Como lidar com pontuação e emoticons fora do padrão de postagens nas redes sociais.

* Compactando vocabulários com stemming e lematização.

* Construidndo uma representação vetorial de uma instrução.

## Tokenizador

* Primeiro passo para um sistema de NLP é um bom vocabulário.

* Aqui iremos estudar algoritmos para separar uma string em palavras em pares, triplas, quadruplas e até quíntuplas. Estes pedaços são chamados de n-grams (n é igual ao tamanho dos pedaços).

* O uso de n-grams permite que sua máquina saiba sobre "ice cream", bem como os "ice" e "cream".

* No processamento de linguagem natural, compor um vetor numérico a partir de texto é um processo de extração de features "com perdas".

* No entanto, o bag-of-words (BOW), retêm o conteúdo de informações do texto suficiente para produzir modelos úteis de máquina.

* A tokenização é um tipo específico de segmentação de documento. A segmentação divide o texto em pedaços ou segmentos menores.

* A segmentação pode incluir a quebra de um documento em parágrafos, parágrafos em sentenças, frases em tokens (geralmente palavras) e pontuação.

* Em compiladores, um tokenizador usado para compilar linguagens de computador é chamado **lexer**.

* O vocubulário (o conjunto de todos os tokens válidos) para uma linguagem de computador é de lexico.

* Para um sistema de NLP, temos (equivalente aos compiladores):

    * Tokenizador: scanner, lexer, analisador lexical.

    * Vocabulário: léxico.

    * Analisador: compilador.

    * token ,termo ou n-gram: token, símbolo ou símbolo terminal.

* A tokenização é a primeira etapa de um pipeline de NLP.

* Objetivo: dividir dados não estruturados (texto em linguagem natural) em chunks de informações discretos.

* Essas contagens de ocorrência de tokens em um documento podem ser usadas diretamente como um vetor que representa este documento.

* Ideia: transformar uma sequeência não estruturada (documento textual) em uma esturtura de dados numérico adqueada para aprendizado de máquina.

* Codificação: A maneira mais simples de tokenizar uma frase é usar espaços em branco em uma string como o "delimitador" das palavras.

In [1]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
sentence.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [2]:
import numpy as np
import pandas as pd

token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))

In [3]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)

print(num_tokens, vocab_size)

10 10


In [4]:
onehot_vector = np.zeros((num_tokens, vocab_size), int)

for i, word in enumerate(token_sequence):
    onehot_vector[i, vocab.index(word)] = 1

In [5]:
pd.DataFrame(onehot_vector, columns=vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


Nesta representação do seu documento, cada linha é um vetor para uma única palavra.

Podemos usar o vetor $[0, 0, 0,	0,	1,	0,	0,	0,	0,	0]$ para representar a palavra "age" no pipeline.

Um "1" em uma coluna indica uma palavra do vocabulário que estava presente nessa posição no documento.

## Bag Of Words

Até aqui criamos uma representação com apneas um único documento (a frase utilizada).

Para ficar mais clara a ideia de bag-of-words precisamos trabalhar com vários documentos para termos um vocubalário (conjunto de palavras) significativamente maior.

In [10]:
sentences = "Thomas Jefferson began building Monticello at the age of 26.\n"
sentences += "Construction was done mstly by local masons and carpenters.\n"
sentences += "He moved into the South Pavillion in 1770.\n"
sentences += "Turning Monticello into a neoclassical masterpiece was Jefferson\n"

corpus = {}

for i, sent in enumerate(sentences.split('\n')):
   corpus[f'sent{i}'] = dict((tok, 1) for tok in sent.split())


df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.,...,moved,into,South,Pavillion,in,1770.,Turning,a,neoclassical,masterpiece
sent0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0,...,1,1,1,1,1,1,0,0,0,0
sent3,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,1,1,1
sent4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Observe o resultado (várias colunas foram omitidas). Cada documento (sentença) possui apenas o valor "1" na posição do vocabulário em que há ocorrência daquela palavra.

* Agora iremos calcular essa sobreposição no pipeline para comparar documentos ou procurar documentos semelhantes, contando o número de tokens sobrepostos usando um produto escalar.